In [ ]:
from brutus.distribute import distribute
from brutus.backends import Lambda

lambda_resource = Lambda()

c = 2

@distribute(backend=lambda_resource)
def foo(a, b):
    """Execute this function"""
    return a + b + c

foo(2, 2)

Traceback (most recent call last):
  File "/home/milesg/anaconda/envs/brutus/lib/python3.5/multiprocessing/queues.py", line 234, in _feed
    obj = ForkingPickler.dumps(obj)
  File "/home/milesg/anaconda/envs/brutus/lib/python3.5/multiprocessing/reduction.py", line 50, in dumps
    cls(buf, protocol).dump(obj)
_pickle.PicklingError: Can't pickle <function foo at 0x7f1a05738a60>: it's not the same object as __main__.foo
